In [1]:
import os
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.models import Model, load_model, Sequential
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization

In [2]:
def define_paths(dir):
    filepaths = []
    labels = []
    folds = os.listdir(dir)
    for fold in folds:
        foldpath = os.path.join(dir, fold)
        filelist = os.listdir(foldpath)
        for file in filelist:
            fpath = os.path.join(foldpath, file)
            filepaths.append(fpath)
            labels.append(fold)
    return filepaths, labels
def define_df(files, classes):
    Fseries = pd.Series(files, name= 'filepaths')
    Lseries = pd.Series(classes, name='labels')
    return pd.concat([Fseries, Lseries], axis= 1)

def create_df(tr_dir, val_dir, ts_dir):
    # train dataframe
    files, classes = define_paths(tr_dir)
    train_df = define_df(files, classes)
    
    # validation dataframe
    files, classes = define_paths(val_dir)
    valid_df = define_df(files, classes)
    # test dataframe
    files, classes = define_paths(ts_dir)
    test_df = define_df(files, classes)
    return train_df, valid_df, test_df

In [3]:
def create_gens(train_df, valid_df, test_df, batch_size):
    img_size = (224, 224)
    channels = 3
    img_shape = (img_size[0], img_size[1], channels)
    ts_length = len(test_df)
    test_batch_size = test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
    test_steps = ts_length // test_batch_size
    def scalar(img):
        return img
    tr_gen = ImageDataGenerator(preprocessing_function= scalar, horizontal_flip= True)
    ts_gen = ImageDataGenerator(preprocessing_function= scalar)
    train_gen = tr_gen.flow_from_dataframe( train_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                        color_mode= 'rgb', shuffle= True, batch_size= batch_size)
    valid_gen = ts_gen.flow_from_dataframe( valid_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                        color_mode= 'rgb', shuffle= True, batch_size= batch_size)
    test_gen = ts_gen.flow_from_dataframe( test_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                        color_mode= 'rgb', shuffle= False, batch_size= test_batch_size)
    return train_gen, valid_gen, test_gen

In [4]:
# Get Dataframes
train_dir = './data-collection/Lung_cancer/train'
test_dir = './data-collection/Lung_cancer/valid'
valid_dir = './data-collection/Lung_cancer/test'
train_df, valid_df, test_df = create_df(train_dir, valid_dir, test_dir)

# Get Generators
batch_size = 40
train_gen, valid_gen, test_gen = create_gens(train_df, valid_df, test_df, batch_size)

Found 613 validated image filenames belonging to 4 classes.
Found 315 validated image filenames belonging to 4 classes.
Found 72 validated image filenames belonging to 4 classes.


In [5]:
train_df

,filepaths,labels
0,./data-collection/Lung_cancer/train\adenocarci...,adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib
1,./data-collection/Lung_cancer/train\adenocarci...,adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib
2,./data-collection/Lung_cancer/train\adenocarci...,adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib
3,./data-collection/Lung_cancer/train\adenocarci...,adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib
4,./data-collection/Lung_cancer/train\adenocarci...,adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib
...,...,...
608,./data-collection/Lung_cancer/train\squamous.c...,squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa
609,./data-collection/Lung_cancer/train\squamous.c...,squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa
610,./data-collection/Lung_cancer/train\squamous.c...,squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa
611,./data-collection/Lung_cancer/train\squamous.c...,squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

# Assuming train_gen and valid_gen are your image data generators

img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = len(list(train_gen.class_indices.keys()))

# Create pre-trained model
base_model = tf.keras.applications.EfficientNetB3(include_top=False, weights="imagenet", input_shape=img_shape, pooling='max')
for layer in base_model.layers:
    layer.trainable = False
model = Sequential([
    base_model,
    BatchNormalization(axis=-1),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(class_count, activation='softmax')
])


In [7]:
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [8]:
# Define filepath to save the best model
filepath = 'lc_v1.h5'

# Create ModelCheckpoint callback to save the best model based on validation accuracy
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [10]:
# Train the model with the added callback
history = model.fit(
    x=train_gen,
    epochs=100,
    verbose=1,
    validation_data=valid_gen,
)

Epoch 1/100
16/16 [==============================] - 26s 2s/step - loss: 0.6646 - accuracy: 0.7586 - val_loss: 1.4602 - val_accuracy: 0.6508
Epoch 2/100
16/16 [==============================] - 26s 2s/step - loss: 0.3886 - accuracy: 0.8581 - val_loss: 0.7973 - val_accuracy: 0.6857
Epoch 3/100
16/16 [==============================] - 25s 2s/step - loss: 0.2750 - accuracy: 0.8956 - val_loss: 0.9067 - val_accuracy: 0.7365
Epoch 4/100
16/16 [==============================] - 25s 2s/step - loss: 0.2432 - accuracy: 0.9103 - val_loss: 0.7787 - val_accuracy: 0.7651
Epoch 5/100
16/16 [==============================] - 26s 2s/step - loss: 0.2193 - accuracy: 0.9201 - val_loss: 0.8705 - val_accuracy: 0.7492
Epoch 6/100
16/16 [==============================] - 26s 2s/step - loss: 0.1643 - accuracy: 0.9380 - val_loss: 0.6077 - val_accuracy: 0.8032
Epoch 7/100
16/16 [==============================] - 25s 2s/step - loss: 0.1868 - accuracy: 0.9315 - val_loss: 0.7309 - val_accuracy: 0.7905
Epoch 8/100
1

In [12]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Average


ts_length = len(test_df)
test_batch_size = test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
test_steps = ts_length // test_batch_size
train_score = model.evaluate(train_gen, steps= test_steps, verbose= 1)
valid_score = model.evaluate(valid_gen, steps= test_steps, verbose= 1)
test_score = model.evaluate(test_gen, steps= test_steps, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

1/1 [==============================] - 2s 2s/step - loss: 0.7709 - accuracy: 0.8611
Train Loss:  0.0301494300365448
Train Accuracy:  0.9750000238418579
--------------------
Validation Loss:  0.6085495352745056
Validation Accuracy:  0.875
--------------------
Test Loss:  0.7708823680877686
Test Accuracy:  0.8611111044883728


In [13]:
from tensorflow.keras.applications import ResNet50

# Assuming train_gen and valid_gen are your image data generators

img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = len(list(train_gen.class_indices.keys()))

# Create pre-trained ResNet50 model
base_model = ResNet50(include_top=False, weights="imagenet", input_shape=img_shape, pooling='max')
for layer in base_model.layers:
    layer.trainable = False
model = Sequential([
    base_model,
    BatchNormalization(axis=-1),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(class_count, activation='softmax')
])

optimizer = Adam()  # Using Adam optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define filepath to save the best model
filepath = 'best_model_resnet.h5'

# Create ModelCheckpoint callback to save the best model based on validation accuracy
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Train the model with the added callback
history = model.fit(
    x=train_gen,
    epochs=100,
    verbose=1,
    validation_data=valid_gen
)

94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/100
16/16 [==============================] - 23s 1s/step - loss: 0.9795 - accuracy: 0.6721 - val_loss: 2.6609 - val_accuracy: 0.6571
Epoch 2/100
16/16 [==============================] - 21s 1s/step - loss: 0.2572 - accuracy: 0.9054 - val_loss: 1.2637 - val_accuracy: 0.7143
Epoch 3/100
16/16 [==============================] - 20s 1s/step - loss: 0.0999 - accuracy: 0.9674 - val_loss: 1.0326 - val_accuracy: 0.7206
Epoch 4/100
16/16 [==============================] - 20s 1s/step - loss: 0.0854 - accuracy: 0.9674 - val_loss: 1.3035 - val_accuracy: 0.6921
Epoch 5/100
16/16 [==============================] - 20s 1s/step - loss: 0.0483 - accuracy: 0.9869 - val_loss: 1.3441 - val_accuracy: 0.7175
Epoch 6/100
16/16 [==============================] - 20s 1s/step - loss: 0.0466 - accuracy: 0.9869 - val_loss: 0.9324 - val_accuracy: 0.7714
Epoch 7/100
16/16 [==============================] - 20s 1s/step - loss: 0.0361 - accurac

In [14]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Average

ts_length = len(test_df)
test_batch_size = test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
test_steps = ts_length // test_batch_size
train_score = model.evaluate(train_gen, steps= test_steps, verbose= 1)
valid_score = model.evaluate(valid_gen, steps= test_steps, verbose= 1)
test_score = model.evaluate(test_gen, steps= test_steps, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

1/1 [==============================] - 2s 2s/step - loss: 0.5517 - accuracy: 0.8889
Train Loss:  2.7924620553676505e-06
Train Accuracy:  1.0
--------------------
Validation Loss:  1.0082703828811646
Validation Accuracy:  0.7749999761581421
--------------------
Test Loss:  0.5516796708106995
Test Accuracy:  0.8888888955116272
